In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

# 프롬프트 템플릿 종류마다 특징과 용도가 있다.... 이것이 요점인듯....
# 일반적인 프로그래밍 내용 같지 않고 관념적인 내용이 들어가서 이해하기가 무척 난해하고 번잡스러운 느낌이 있다
# 기본적인 흐름 만이라도 정확하게 이해하고자 해야 한다.
# 이해도가 떨어지면 시간이 소비되고 많은 시간을 소비하면 다른 부분들에서 소홀 해 질 수 있으니 밸런스를 잘 맞추자.... 추후 보강할 기회는 분명히 올것이다
# jason 을 이용한 샘플, 템플릿 등의  db 화, 특정 생플 수와 선택, 동일한 질문의 캐싱 처리, 토큰의 확인과 모델, 환경 설정 저장과 변경 등이 강의 내용에 있다.

# FewShotChatMessagePromptTemplate 을 사용한다 길어서 뭔가 더 좋을것 같다

chat = ChatOpenAI(
    model="gpt-3.5-turbo",                                 # 혹시나 해서 모델을 지정해 준다
    temperature=0.1,                                       # 성능 쯤으로 생각하자
    streaming=True,                                        # 체인의 호출에 따른 내용을 스트리밍 즉 과정을 보여준다
    callbacks=[StreamingStdOutCallbackHandler(),]          # 한글짜씩 보여달라고 요청한다
    )

examples = [                                               # 샘플의 형식이다 """ 이렇게 세개를 쓰면 여러줄을 쓸 수 있다.... 리스트 안에 딕셔너리 구조이다 마우스 올려보면 나옴
    {
        "movie": "아이언맨",
        "answer": """
        감독 : Jon Favreau
        주연: 로버트 다우니 주니어
        장르: SF
        간단한 줄거리: 토니 스타크가 악당을 깨부수는 영화
        20자 영화평: 1편이 제일 재미있다. 다른건 꽝!
        """
    },
    {
        "movie": "기생충",
        "answer": """
        감독 : 봉준호, 
        주연: 송강호, 장혜진, 최우식, 박소담, 고 이선균, 조여정, 정지소, 정현준, 이정은
        장르: 서스펜스, 블랙코미디
        줄거리: 가난하고 비천한 가족이 부자집에 몰래 속이고 접근하여 기생충 처럼 산다는 내용
        20자 영화평: 냄새 때문에 열받아서.... 이것 빼고는 다 좋다...
        """
    },
    {
        "movie": "록키 호러 픽쳐 쇼",
        "answer": """
        감독 : 짐 셔먼, 
        주연: 팀 커리, 수잔 서랜든, 베리 보스트윅, 조나단 아담스
        장르: 컬트, 뮤지컬
        줄거리: 신혼 부부가 어떤 성 같은데 들어가서 버라이어티한 모험을 격고 탈출하는 영화
        20자 영화평: 어처구니가 없어지는데 기가 막힌다....
        """
    }
]

# 오로지 example_prompt_form  을 만들기 위한 템플릿 이다 강의내용은 덮어쓰기 형식으로 되어 있기 때문에 초보자는 헤깔릴 위험이 있다(이러한 습관은 프로그래머의 종특인 것인가) 그래서 수정했다
example_prompt = ChatPromptTemplate.from_messages(                  # chat 프롬프트템플릿을 만든다 모든게 프롬프트를 어떻게 만드느냐에 따라서 결과물이 달라진다.. 이 템플릿은 샘플로 사용할 예정이 되겠다. 
    [                                                               # 구조는 [(str,str),(str,str)]  이렇게 되어 있다 헤깔림... 
        ("human", "{movie} 에 대하여 알려줘"),                       # 그니깐 샘플에 있는 컨트리를 휴먼이 입력하면 (이 폼과 다른 폼으로 질문하면 어떨찌 모르겠는데 해보고 적겠다) (해 봤더니 똑 같다 좀더 간단히 해도 된다)
        ("ai", "{answer}")                                          # ai 가 샘플처럼 답변하는 다는 것을 학습? 혹은 양식을 알려주기 위한 폼이 되겠다.. 오로지 샘플의 형식을 위한 템플릿 구조라 보면 되겠다
    ]
)

# 이게 이름이 위엣거랑 같아서 헤깔리게 만드는 요인인것 같다 그래서 이름을 바꿨다 이렇게 하면 용도가 확실히 구분될 수 있을것 같다.  FewShotChatMessagePromptTemplate 가 폼을 학습시키는 특징이 있는것이로다...!!! 레퍼런스를 주면 이해를 하다니!!! 듣도보도 못한 내용이다....
example_prompt_form = FewShotChatMessagePromptTemplate(             # 위에서 만든 chat 프롬프트의 형식을 지정하는것으로 생각하면 될것 같다.... 뭔가 토큰이 적게 드는 방식이라고 생각된다. 
    examples=examples,                                              # 질문과 답변의 샘플이다. 이걸 템플릿에 입력해준다 
    example_prompt=example_prompt                                   # 윗줄의 샘플을 가지고 만만드는 프롬프트 템플릿의 구조 이다 단순하게 물어보면 답변하는 형식이다
    
)

final_prompt = ChatPromptTemplate.from_messages(                                                                     # 최종 프롬프트의 형태이다 위에서 설명한 대로 [(str,str),(str,str)] 형태로 되어 있다
    [
        ("system", "너는 한국의 이동진 영화 평론가야. 너는 술을 마시면 진심을 담아서 짧게 이야기 하는 편이야"),             # 자아를 입히는 과정이다
        example_prompt_form,                                                                                         # 이넘아는 오로지 형식을 참고하는 템플릿 이다.... form 이 가지고 있는 샘플과 샘플 프롬프트를 참고 한다   
        ("human", "{movie} 에 대하여 알려줘")                                                                         # 이건 입력이다 일부러 다르게 해 보기도 하고 같게 해 보기도 하고 하면 좋을것 같다(해봤더니 비슷하다 우선순위 같은것이 있을꺼 같기도 하고 담에 해 보자)
      ]  
)

chain = final_prompt | chat                                                                 # 파이널 프롬프트를 챗으로 넘기는 체인 구조를 만든다

chain.invoke({"movie": "록키 호러 픽쳐 쇼"})                                                 # 파이널 프롬프트에 필요한 딕셔너리를 너어주고 체인 구조를 호출한다

# 샘플에 있는 영화를 검색 했더니 내가 쓴것과는 다르게 나옴 지가 뭘 안다고....



        감독 : 짐 셔먼
        주연: 팀 커리, 수잔 서랜든
        장르: 뮤지컬, 호러, 코미디
        줄거리: 이상한 외계인들이 등장하는 성에 갇힌 커플의 이야기
        20자 영화평: 미친듯한 노래와 춤, 기묘한 스토리. 호러와 코미디의 완벽한 조합!

AIMessageChunk(content='\n        감독 : 짐 셔먼\n        주연: 팀 커리, 수잔 서랜든\n        장르: 뮤지컬, 호러, 코미디\n        줄거리: 이상한 외계인들이 등장하는 성에 갇힌 커플의 이야기\n        20자 영화평: 미친듯한 노래와 춤, 기묘한 스토리. 호러와 코미디의 완벽한 조합!')

In [15]:
chain.invoke({"movie": "스파이더맨"})  


        감독 : 산 레이미, 마크 웹, 존 왓츠, 존 왓츠, 존 왓츠
        주연: 토비 맥과이어, 앤드류 가필드, 톰 홀랜드
        장르: 슈퍼히어로, 액션, 모험
        줄거리: 젊은 피터 파커가 스파이더맨으로 변신하여 범죄와 싸우는 이야기
        20자 영화평: 토비 맥과이어가 짱! 앤드류 가필드도 괜찮고, 톰 홀랜드는 어떨지...

AIMessageChunk(content='\n        감독 : 산 레이미, 마크 웹, 존 왓츠, 존 왓츠, 존 왓츠\n        주연: 토비 맥과이어, 앤드류 가필드, 톰 홀랜드\n        장르: 슈퍼히어로, 액션, 모험\n        줄거리: 젊은 피터 파커가 스파이더맨으로 변신하여 범죄와 싸우는 이야기\n        20자 영화평: 토비 맥과이어가 짱! 앤드류 가필드도 괜찮고, 톰 홀랜드는 어떨지...')

In [9]:
chain.invoke({"movie": "기생충"})  


        감독 : 봉준호
        주연: 송강호, 이선균, 조여정, 최우식
        장르: 블랙 코미디, 스릴러
        줄거리: 가난한 가족이 부자 가정에 침투하며 벌어지는 사건들을 그린 작품
        20자 영화평: 사회 비판적 메시지를 담은 봉준호 감독의 작품. 연출과 연기 모두 훌륭하다.

AIMessageChunk(content='\n        감독 : 봉준호\n        주연: 송강호, 이선균, 조여정, 최우식\n        장르: 블랙 코미디, 스릴러\n        줄거리: 가난한 가족이 부자 가정에 침투하며 벌어지는 사건들을 그린 작품\n        20자 영화평: 사회 비판적 메시지를 담은 봉준호 감독의 작품. 연출과 연기 모두 훌륭하다.')

In [14]:
chain.invoke({"movie": "에일리언"})  


        감독 : 리들리 스콧
        주연: 시고니 위버, 톰 스콧, 존 허트, 이안 홈, 헤리 딘 스탠튼
        장르: SF, 호러
        줄거리: 우주선에서 외계 생명체에 의해 습격당하는 스페이스 호러물
        20자 영화평: 외계 생명체가 무서워서 눈을 뗄 수 없는 명작!

AIMessageChunk(content='\n        감독 : 리들리 스콧\n        주연: 시고니 위버, 톰 스콧, 존 허트, 이안 홈, 헤리 딘 스탠튼\n        장르: SF, 호러\n        줄거리: 우주선에서 외계 생명체에 의해 습격당하는 스페이스 호러물\n        20자 영화평: 외계 생명체가 무서워서 눈을 뗄 수 없는 명작!')